In [1]:
## Milestone 1: Smart Tracking System Blockchain Ledger

In [2]:
!pip install web3

In [4]:
## To verify if the IOT sensor data from CSV will load
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("smart_environment_data_Official.csv")


# Display the first few rows
print(df.head())

          timestamp sensor_id     co2_ppm  pm25_ugm3  temperature_c  \
0   07/05/2025 4:14   SENS762  433.043819  35.411776           30.7   
1  06/05/2025 23:48   SENS414  378.758719  19.912831           34.1   
2  06/05/2025 13:46   SENS874  411.669284  39.356963           31.2   
3   07/05/2025 2:09   SENS903  475.838638  29.126293           28.7   
4   06/05/2025 8:35   SENS114  396.233177  40.948456           32.0   

   humidity_percent  soil_moisture_percent  water_ph  turbidity_ntu  
0              64.6                   28.8      6.88           1.82  
1              83.7                   29.3      8.18           4.00  
2              85.1                   22.7      6.89           1.34  
3              76.8                   15.6      7.85           1.13  
4              81.7                   27.9      7.73           0.04  


In [5]:
#Establishment of Web 3 connection to the Ganache RPC URL
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


In [11]:
# Replace with actual contract address from Remix
contract_address = "0xF722Ecca72f1F3C33FBd7c1Bf8FCd29BA4BcE883"


# Paste the ABI from Remix
abi = [
    {
        'inputs': [],
        'stateMutability': 'nonpayable',
        'type': 'constructor'
    },
    {
        'anonymous': False,
        'inputs': [
            {'indexed': False, 'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'indexed': False, 'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'name': 'DataRetrieved',
        'type': 'event'
    },
    {
        'anonymous': False,
        'inputs': [
            {'indexed': False, 'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'indexed': False, 'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'name': 'DataStored',
        'type': 'event'
    },
    {
        'inputs': [
            {'internalType': 'string', 'name': '_sensorId', 'type': 'string'},
            {'internalType': 'string', 'name': '_dataType', 'type': 'string'},
            {'internalType': 'string', 'name': '_dataValue', 'type': 'string'}
        ],
        'name': 'storeData',
        'outputs': [],
        'stateMutability': 'nonpayable',
        'type': 'function'
    },
    {
        'inputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'name': 'dataRecords',
        'outputs': [
            {'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [
            {'internalType': 'uint256', 'name': 'index', 'type': 'uint256'}
        ],
        'name': 'getRecord',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'},
            {'internalType': 'string', 'name': '', 'type': 'string'},
            {'internalType': 'string', 'name': '', 'type': 'string'},
            {'internalType': 'string', 'name': '', 'type': 'string'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'getTotalRecords',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'MAX_ENTRIES',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'owner',
        'outputs': [
            {'internalType': 'address', 'name': '', 'type': 'address'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    }
]
  # Replace with your contract ABI


# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set the default sender address (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0xF722Ecca72f1F3C33FBd7c1Bf8FCd29BA4BcE883


In [12]:
import time


def send_iot_data(sensor_id, data_type, data_value):
    """Sends IoT data to the deployed smart contract"""
    txn = contract.functions.storeData(sensor_id, data_type, data_value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
   
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {data_type} - {data_value}, Txn Hash: {receipt.transactionHash.hex()}")

In [13]:
# List of all metrics you want to send
metrics = [
    'co2_ppm',
    'pm25_ugm3',
    'temperature_c',
    'humidity_percent',
    'soil_moisture_percent',
    'water_ph',
    'turbidity_ntu'
]

# Loop through each row of the DataFrame
for _, row in df.iterrows():
    sensor_id = str(row['sensor_id'])  # Extract the sensor ID
    for metric in metrics:
        data_type = metric  # e.g., 'co2_ppm'
        data_value = str(row[metric])  # e.g., '412.5'
        
        # Send the data to the smart contract
        send_iot_data(sensor_id, data_type, data_value)
        
        # Wait for 1 second to prevent flooding the network
        time.sleep(1)

✅ Data Stored: co2_ppm - 433.0438194, Txn Hash: dcfe9916180fa0d9b7a5910edff504996f6999997997172db039e5580827d294
✅ Data Stored: pm25_ugm3 - 35.41177588, Txn Hash: a8723c985a4cc1b5d6017f0bbf7c6cc3331e7d5c7c322959fdaec083a37a7624
✅ Data Stored: temperature_c - 30.7, Txn Hash: 072e2cec4460f35e750554400b95de50aeeb31c67cc171cc23b2a61a410df5b9
✅ Data Stored: humidity_percent - 64.6, Txn Hash: 1d66ee90d6ffc0d595742cc16d3e68b3d2b78e48a98f427a074a56f73c30af98
✅ Data Stored: soil_moisture_percent - 28.8, Txn Hash: 1b4fb746239de48358a260fd06ccfdc023f74d1ed7a908b5d3b922c37035440e
✅ Data Stored: water_ph - 6.88, Txn Hash: d5e2d5d20efddbd2c5cb817400def742c4cda06c7ff5865f16782d181e5fcfaa
✅ Data Stored: turbidity_ntu - 1.82, Txn Hash: 9f38bc3c19a65ccd834d42f6687272525a914e349a1ba2791c6e29585c33ed74
✅ Data Stored: co2_ppm - 378.758719, Txn Hash: 6589682800a655d7a3e06206758ae91dcccfa550d8c7450b4773a8c58f52bb42
✅ Data Stored: pm25_ugm3 - 19.91283145, Txn Hash: babe3913a8d51bf7a20230b4db39de895bdab5dac728

In [9]:
print(df.columns)

Index(['timestamp', 'sensor_id', 'co2_ppm', 'pm25_ugm3', 'temperature_c',
       'humidity_percent', 'soil_moisture_percent', 'water_ph',
       'turbidity_ntu'],
      dtype='object')


In [14]:
## Getting the total records
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 700


In [15]:
## Retrieving and Printing a specific record (0)
record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

First Stored Record: [1748976119, 'SENS762', 'co2_ppm', '433.0438194']
